In [1]:
from bs4 import BeautifulSoup  
import requests
import pandas as pd 

In [2]:
Genres=["Action","Action-Adventure","Adventure","Board+Game","Education","Fighting","Misc","MMO","Party","Music","Platform","Puzzle","Racing","Role-Playing","Sandbox","Shooter","Simulation","Sports","Strategy","Visual+Novel"]

def Get_Games_Part1(genres,sfilter):
    #list for every column
    Title=[] 
    Platforms=[] 
    Publisher=[] 
    Total_sales=[]
    NA_Sales=[] 
    PAL_Sales=[] 
    Japan_Sales=[]
    Release_date=[] 
    Genre_game=[] 
    
    for genre in genres:
        url1=f"https://www.vgchartz.com/games/games.php?page=1&results=200&genre={genre}&order=TotalSales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=0"
        html1=requests.get(url1)
        soup1=BeautifulSoup(html1.content,"html.parser")
        
        pages=int(soup1.findAll("span")[-3].find("a").attrs["href"].split("=")[1].split("&")[0]) #all pages

        
        games_tags=[] 
        for p in range(0, pages):
            url=f"https://www.vgchartz.com/games/games.php?page={p}&results=200&genre={genre}&order=TotalSales&ownership=Both&direction=DESC&showtotalsales=1&shownasales=1&showpalsales=1&showjapansales=1&showothersales=1&showpublisher=1&showdeveloper=0&showreleasedate=1&showlastupdate=0&showvgchartzscore=1&showcriticscore=0&showuserscore=0&showshipped=0"
            html=requests.get(url)
            soup=BeautifulSoup(html.content,"html.parser")
            games_tags.extend(soup.findAll("tr",sfilter))
        
        for tag in games_tags:
            Title.append(tag.findAll("td")[2].text.strip())
            Platforms.append(tag.findAll("td")[3].find("img").attrs["alt"])
            Publisher.append(tag.findAll("td")[4].text.strip())
            Total_sales.append(tag.findAll("td")[6].text )
            NA_Sales.append(tag.findAll("td")[7].text)
            PAL_Sales.append(tag.findAll("td")[8].text)
            Japan_Sales.append(tag.findAll("td")[9].text )
            Release_date.append(tag.findAll("td")[11].text.strip() )
            Genre_game.append(genre)
            
            #Dictionary containing all the games   
            dic_games={"Title":Title,
                       "Platforms":Platforms,
                       "Publisher":Publisher,
                       "Genre": Genre_game,
                       "Total_sales":Total_sales,
                       "NA_Sales":NA_Sales,
                       "PAL_Sales":PAL_Sales,
                       "Japan_Sales":Japan_Sales,
                       "Release_date":Release_date}
            
    return dic_games



In [3]:
def df_games(dic):
    df=pd.DataFrame(dic)
    #Deleting the m from the coloumns
    df["Total_sales"]=df["Total_sales"].apply(lambda x :x.split("m")[0]) 
    df["NA_Sales"]=df["NA_Sales"].apply(lambda x :x.split("m")[0]) 
    df["PAL_Sales"]=df["PAL_Sales"].apply(lambda x :x.split("m")[0]) 
    df["Japan_Sales"]=df["Japan_Sales"].apply(lambda x :x.split("m")[0])
    
    #Formating the date
    df["Release_date"]=df["Release_date"].apply(lambda x:" ".join(x.split("th")))
    df["Release_date"]=df["Release_date"].apply(lambda x:" ".join(x.split("st"))) 
    return df

In [ ]:
sf1={"style":"background-image:url(../imgs/chartBar_large.gif); height:70px"}
sf2={"style":"background-image:url(../imgs/chartBar_alt_large.gif); height:70px"}
df_tr_1=df_games(Get_Games_Part1(Genres,sf1))
df_tr_2=df_games(Get_Games_Part1(Genres,sf2))
df_all_games=pd.concat([df_tr_1,df_tr_2])#concating two dataframes
df_all_games.to_csv("Games_Data.csv",index=False)
#Concatenating the Two Dataframe into one 

